In [1]:
import psycopg 
conn_info = "dbname=rag_chatbot user=postgres password=Postgres1925 host=localhost port=5433" # création de la connexion
conn = psycopg.connect(conn_info) 
print("Connexion OK")

Connexion OK


In [2]:
with conn.cursor() as cur:
    cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
    print("Extension pgvector créée")

Extension pgvector créée


In [3]:
with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS embeddings (
            id SERIAL PRIMARY KEY,
            corpus_text TEXT NOT NULL,
            embedding VECTOR(768)  
        );
    """)
    conn.commit()

print("Table embeddings créée !")


Table embeddings créée !


In [4]:
from sentence_transformers import SentenceTransformer

# Charger un modèle Hugging Face
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

def embed_text(text: str):
    """Retourne un embedding Hugging Face."""
    embedding = model.encode(text, convert_to_numpy=True)
    return embedding.tolist()   # convertir pour PostgreSQL

c:\Users\asusm\Desktop\Chatbot-RAG\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
texte = "Ceci est un exemple de document"
vecteur = embed_text(texte)
print(len(vecteur))

768


In [6]:
import psycopg
from pgvector.psycopg import register_vector
import glob 
# enregistrer le type vector avec la connexion
register_vector(conn)

for filepath in glob.glob("../data/*.txt"):
    with open(filepath, "r", encoding="utf-8", errors="ignore") as f:
        texte = f.read()
        emb = embed_text(texte)  # retourne déjà une liste de floats

        with conn.cursor() as cur:
            cur.execute("""
                INSERT INTO embeddings (corpus_text, embedding)
                VALUES (%s, %s)
            """, (texte, emb))  # ici emb est une liste
    conn.commit()


In [7]:
with conn.cursor() as cur:
    cur.execute("SELECT id, corpus_text FROM embeddings LIMIT 5;")
    print(cur.fetchall())

[(1, '<01> hotesse\n     h: U B S bonjour\n<02> client\n     c: oui bonjour e j\'appelle je sais pas si j\'appelle au bon endroit e\n<03> hotesse+client\n     h: je vous coute\n     c: c\'est pour\n<04> client\n     c: e c\'est pour savoir si la fac pendant l\'t e a des professeurs ou des des gens qui font des stages de de perfectionnement en anglais et en espagnol\n<05> hotesse\n     h: e ce serait pour vous vous souhaiteriez\n<06> hotesse+client\n     h: non\n     c: non non c\'est pas pour moi\n<07> client\n     c: ce serait pour ma fille\n<08> hotesse+client\n     h: oui\n     c: mais bon elle est\n<09> client\n     c: en seconde et\n<10> hotesse+client\n     h: et elle souhaiterais se perfectionner\n     c: bon elle va passer en premire mais\n<11> hotesse\n     h: en anglais ou en espagnol pendant l\'t\n<12> client\n     c: ouais c\'est \n<13> hotesse\n     h: oui alors e la fac de e de lettre et de langues se trouve  Lorient donc il faudrait plutt  voir avec Lorient pour e savoir

In [8]:
with conn.cursor() as cur:
    cur.execute("SELECT id, corpus_text, embedding FROM embeddings LIMIT 5;")
    for row in cur.fetchall():
        print(row[0], row[1], row[2][:10])

1 <01> hotesse
     h: U B S bonjour
<02> client
     c: oui bonjour e j'appelle je sais pas si j'appelle au bon endroit e
<03> hotesse+client
     h: je vous coute
     c: c'est pour
<04> client
     c: e c'est pour savoir si la fac pendant l't e a des professeurs ou des des gens qui font des stages de de perfectionnement en anglais et en espagnol
<05> hotesse
     h: e ce serait pour vous vous souhaiteriez
<06> hotesse+client
     h: non
     c: non non c'est pas pour moi
<07> client
     c: ce serait pour ma fille
<08> hotesse+client
     h: oui
     c: mais bon elle est
<09> client
     c: en seconde et
<10> hotesse+client
     h: et elle souhaiterais se perfectionner
     c: bon elle va passer en premire mais
<11> hotesse
     h: en anglais ou en espagnol pendant l't
<12> client
     c: ouais c'est 
<13> hotesse
     h: oui alors e la fac de e de lettre et de langues se trouve  Lorient donc il faudrait plutt  voir avec Lorient pour e savoir si ils organisent des stages mais en tou

In [9]:
from pgvector.psycopg import Vector

def search_similar(query_embedding, top_k=5):
    with conn.cursor() as cur:
        vector_value = Vector(query_embedding)
        cur.execute("""
            SELECT id, corpus_text, embedding <-> %s AS distance
            FROM embeddings
            ORDER BY distance
            LIMIT %s
        """, (vector_value, top_k))
        return cur.fetchall()

conn.rollback()  # Important si transaction précédente échouée

# Exemple : texte de la question
user_query = "Bonjour"

# Générer l'embedding pour la question
query_emb = embed_text(user_query)  # embed_text retourne une liste de floats

# Maintenant tu peux appeler ta fonction search_similar
results = search_similar(query_emb)
for r in results:
    print("ID:", r[0], "Distance:", r[2], "Texte:", r[1][:100], "…")


ID: 188 Distance: 1.0130555736138578 Texte: <01> hotesse
     h: U B S bonjour
<02> client
     c: oui bonjour madame j'aurais voulu avoir des r …
ID: 106 Distance: 1.0130555736138578 Texte: <01> hotesse
     h: U B S bonjour
<02> client
     c: oui bonjour madame j'aurais voulu avoir des r …
ID: 65 Distance: 1.0130555736138578 Texte: <01> hotesse
     h: U B S bonjour
<02> client
     c: oui bonjour madame j'aurais voulu avoir des r …
ID: 24 Distance: 1.0130555736138578 Texte: <01> hotesse
     h: U B S bonjour
<02> client
     c: oui bonjour madame j'aurais voulu avoir des r …
ID: 147 Distance: 1.0130555736138578 Texte: <01> hotesse
     h: U B S bonjour
<02> client
     c: oui bonjour madame j'aurais voulu avoir des r …


In [13]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.eval()

# Prompt QA
texts_similaires = [r[1] for r in results[:3]]
context = "\n".join(texts_similaires)
question = user_query

prompt = f"Answer the question based on the context below:\nContext: {context}\nQuestion: {question}"

inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)

with torch.no_grad():
    outputs = model.generate(
        inputs["input_ids"],
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7
    )

answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(answer)


c:\Users\asusm\Desktop\Chatbot-RAG\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\asusm\.cache\huggingface\hub\models--google--flan-t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTT

Hi, Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Is this your name? Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. What is this name? Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Welcome. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello. Hello
